In [27]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [28]:
df = pd.read_pickle("../data/bikes_registrados_estand_encod.pkl")
df.head()

,año,mes,festivo,dia,meteo,temperatura,humedad,velo_viento,usu_registrados
0,0,0,0,0,1,-0.486274,0.858854,-0.208909,654
1,0,0,1,0,1,-0.425690,0.331548,0.687550,670
2,0,0,1,1,2,-0.949970,-0.912000,0.685210,1229
3,0,0,1,1,2,-0.938563,-0.176088,-0.210435,1454
4,0,0,1,1,2,-0.853992,-0.913519,0.060294,1518


In [29]:
X = df.drop("usu_registrados", axis = 1)
y = df["usu_registrados"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 15)

arbol = DecisionTreeRegressor(random_state =15)

arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=15)

In [30]:
with open ("../data/dt_registrados.pkl", "wb") as f:
    pickle.dump(arbol, f)

In [31]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [32]:
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [33]:
max_features = np.sqrt(len(x_train.columns))
max_features

2.8284271247461903

In [34]:
print(arbol.tree_.max_depth)

20


In [41]:
param = {"max_depth": [2, 3, 4],
         "max_features": [2, 3, 4],
         "min_samples_split": [5, 10, 25],
         "min_samples_leaf": [5, 10, 20]} 

In [42]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(), 
                  param_grid= param,
                  cv=10,
                  verbose=0,
                  return_train_score = True,
                  scoring="neg_mean_squared_error")

In [43]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 3, 4], 'max_features': [2, 3, 4],
                         'min_samples_leaf': [5, 10, 20],
                         'min_samples_split': [5, 10, 25]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [44]:
with open ("../data/dt_registrados_mejor_modelo.pkl", "wb") as f:
    pickle.dump(gs, f)

In [45]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=4, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [46]:
dt_results = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "DT_registrados_estad_encode")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,588.226027,756354.746575,869.68658,0.718272,test,DT_registrados_estad_encode
1,0.000000,0.000000,0.00000,1.000000,train,DT_registrados_estad_encode
